In [3]:
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_id = stopwords.words('indonesian') + list(punctuation)

# Import Data

In [4]:
df = pd.read_csv('data/kompas.csv')
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_id)
tfid_matrix = tfidf.fit_transform(df.teks)

C:\Users\ASUS\miniconda3\envs\jcopml\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# TF-IDF Similarity -> Document Similarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
sim = cosine_similarity(tfid_matrix[0], tfid_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [14]:
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

## Most similar

In [16]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [17]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [19]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

## Least similar

In [20]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

In [21]:
df.teks[1131][:200]

'Merpati Mendarat Darurat\n\nPesawat Penumpang CASA 212 Merpati Ambon-Saumlaki, Kamis (18/4) pukul 16.30, mendarat darurat di Desa Rumah Lewang Besar, Kecamatan Pulau-pulau Babar, Kabupaten Maluku Tengga'

In [22]:
df.teks[1593][:200]

'Kapal Kargo Tenggelam di Sumenep, 11 Orang Hilang\n\nSebuah kapal kargo yang belum diketahui namanya tenggelam di perairan Masa Lembu, Sumenep, Madura, Rabu (6/2). Akibat musibah itu, 11 orang hilang, s'

# Keyword Extraction

In [24]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [25]:
tfid_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [39]:
sorted_tfidf = tfid_matrix[0].toarray().argsort()
sorted_tfidf

array([[274212, 365469, 365468, ..., 386379, 436652, 169219]], dtype=int64)

In [40]:
vocab[169219]

'ginandjar'

In [41]:
vocab[436652]

'putusan'

In [42]:
vocab[386379]

'penahanan'

In [45]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()
    
    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [46]:
text = '''
Jakarta, CNN Indonesia -- Anggota Komisi VIII DPR dari fraksi Partai Keadilan Sejahtera (PKS), Bukhori Yusuf menolak aturan baru Kementerian Agama (Kemenag) soal penggunaan pengeras suara di masjid dan musala.
Bukhori menyarankan agar hal itu mestinya bisa diserahkan kepada masyarakat secara tradisi dan musyawarah.

"Biarkan masyarakat yang mengatur secara tradisi atau musyawarah," kata dia kepada CNNIndonesia.com, Senin (21/2).


Bukhori menilai, aturan Kemenag soal penggunaan pengeras suara di masjid dan musala terlalu mencampuri ranah teknis peribadatan.

Lihat Juga :

DMI Dukung Aturan Baru Menag Sepiker Masjid: Tingkat Bising Tinggi
Padahal, setiap kelompok masyarakat di suatu wilayah memiliki tradisi yang berbeda, dan hal itu bisa diatur secara tradisi oleh mereka.

"Menurut saya kemenag tidak perlu mengatur hal-hal yang sangat teknis tentang masalah ibadah utamanya penggunaan speker untuk adzan, pengajian, maupun lainnya di masyarakat, karena hal itu di setiap kampung yang satu dengan lainnya tidak sama," kata dia.

Aturan mengenai penggunaan spiker masjid dan musala sebelumnya diteken Menag Yaqut lewat surat edaran (SE) Nomor 05 tahun 2022 yang mengatur tentang penggunaan pengeras suara di masjid dan musala.

Lihat Juga :

SE Menag: Sepiker Luar Masjid Saat Takbiran Sampai Jam 10 Malam
Salah satu poin penting yang diatur dalam edaran itu yakni volume pengeras suara masjid/musala paling besar 100 dB atau desibel dengan suara tidak sumbang.

Menurut Yaqut, penggunaan pengeras suara di masjid dan musala memang kebutuhan bagi umat Islam sebagai salah satu media syiar Islam di tengah masyarakat.

Namun, di sisi lain, masyarakat Indonesia terdiri dari beragam agama, keyakinan, dan latar belakang sehingga perlu upaya merawat persaudaraan dan harmoni sosial.

"Pedoman diterbitkan sebagai upaya meningkatkan ketenteraman, ketertiban, dan keharmonisan antarwarga masyarakat," ujar Yaqut.



Baca artikel CNN Indonesia "PKS Tolak Aturan Menag soal Sepiker Masjid: Biar Warga yang Atur" selengkapnya di sini: https://www.cnnindonesia.com/nasional/20220221175411-32-762069/pks-tolak-aturan-menag-soal-sepiker-masjid-biar-warga-yang-atur.

Download Apps CNN Indonesia sekarang https://app.cnnindonesia.com/
'''

In [47]:
extract_keywords_tfidf(text, tfidf)

['masjid',
 'suara masjid',
 'pengeras',
 'pengeras suara',
 'menag',
 'penggunaan',
 'suara',
 'aturan',
 'cnn',
 'tradisi']